In [ ]:
import sys
import time
import numpy as np
import pandas as pd
import itertools
import plotly.express as px
import plotly.graph_objects as go
import datetime

from sklearn.covariance import GraphicalLasso as sk_GL
from sklearn.covariance import empirical_covariance
from sklearn import set_config
set_config(print_changed_only=False)

from gglasso.solver.single_admm_solver import ADMM_SGL
from gglasso.solver.single_admm_solver import block_SGL
from gglasso.helper.data_generation import time_varying_power_network, group_power_network, sample_covariance_matrix
from gglasso.helper.model_selection import single_grid_search
from benchmarks import models_to_dict, sklearn_time_benchmark, admm_time_benchmark, model_solution
from benchmarks import network_generation, hamming_distance, sparsity_benchmark, time_benchmark, dict_to_dataframe

from regain.covariance import GraphicalLasso as rg_GL

In [ ]:
S_dict, X_dict, Theta_dict = network_generation(p_list=[300, 500, 1000], N_list=[600, 1000, 2000], K=5, M=2)

In [ ]:
time_dict, accuracy_dict, Z_dict = time_benchmark(X_dict=X_dict, S_dict=S_dict, Z_model="sklearn",
                                                  lambda1=0.1, tol_list=[1e-4, 1e-6], rtol_list=[1e-2, 1e-4],
                                                  enet_list=[1])

In [ ]:
sparsity = sparsity_benchmark(Theta_dict=Theta_dict, Z_dict=Z_dict, t_rounding=1e-4)

In [ ]:
df = dict_to_dataframe(times=time_dict, acc_dict=accuracy_dict, spars_dict=sparsity)
df.head()

In [ ]:
fig = px.scatter(df, x="time", y="accuracy", text = "name", color= "method", 
              log_y = True, facet_col='p', facet_col_wrap=3,
              labels={
                     "time": "Time, s",
                     "accuracy": "Log_distance",
                     "method": "method"
                 },
              template = "plotly_white",
              title="Log-distance between Z and Z' with respect to ADMM convergence rates")
fig.update_traces(mode='markers+lines')
fig.show()

In [ ]:
fig = px.scatter(df, x="method", y="hamming", text = "name", color= "method", 
              log_y = True, facet_col='p', facet_col_wrap=2,
              labels={
                     "time": "Time, s",
                     "accuracy": "Log_distance",
                     "method": "method"
                 },
              template = "plotly_white",
              title="Log-distance between Z and Z' with respect to ADMM convergence rates")
fig.update_traces(mode='markers+lines')
fig.show()